<a href="https://colab.research.google.com/github/BearsOnMars/baby-alpha/blob/main/Scraping_Financial_Data_from_Yahoo_Finance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Scraping data using 'hidden APIs' method and 'extracting embedded json formatted data' method. These can be easily converted to a python dictionary. Standard libraries are used

In [ ]:
import re
import json
import csv
from io import StringIO
# Installing Beautiful Soup and Requests package. Note the exclamation mark at the beginning
# In Google Colab, we can run a console command by starting with a ! followed by the command
!pip install bs4
!pip install requests
from bs4 import BeautifulSoup
import requests

Going to Yahoo Finance now, entering relevant stock ticker in search. Data can be extracted for any tab i.e. statistics, historical data, financials, options. went and searched for The Coca-Cola company. copying the url for the tabs we are interested in scraping. In place of the stock ticker symbol such as KO for Coca-Cola in these similar looking urls, if we use {}, it becomes easier at later stages to insert any new symbol with string formatting at a later stage.

In [ ]:
url_stats = 'https://finance.yahoo.com/quote/{}/key-statistics?p={}'
url_profile = 'https://finance.yahoo.com/quote/{}/profile?p={}'
url_financials = 'https://finance.yahoo.com/quote/{}/financials?p={}'

In [ ]:
# Assigning KO to stock Coca Cola so as to look it up on Yahoo finance for the ticker
stock = 'KO'

Scraping Financial Statements. Using git request(?) even data which can not be seen in the website can be downloaded. Using financials url template to request the stock data using request library. Response is assigned to a response variable

In [ ]:
response = requests.get(url_financials.format(stock, stock))

In [ ]:
# HTML data is parsed using BeautifulSoup and default HTML parser
soup = BeautifulSoup(response.text, 'html.parser')

Going back to the webpage now to look at raw data by right-click 'view page source'. The webpage is dynamically loading data from the javascript function contained within the script tag. The comment /* -- Data --*/ contains all data which is dynamically being loaded on to the page. Since, the data is embedded in a javascript function, we need to get the script tag first, then extract the contents from it, chop the edges off so that we are just left with the json formatted string, which can then be converted to a python dictionary. There is no unique id tag for the script, so we need to use a text pattern with regular expressions

In [ ]:
pattern = re.compile(r'\s--\sData\s--\s')

using this pattern now find the script element that has text which matches this pattern. return the contents of this element as a list. grab the first and only item in the list


In [ ]:
script_data = soup.find('script', text=pattern).contents[0]

In [ ]:
# Printing the beginning and the ending 500 characters of the script data
# Notice the colon at start in beginning 
script_data[:500]

In [ ]:
# Notice the negative sign and the colon at the end while printing ending
script_data[-500:]

The problem now is that we have this javascript code wrapping the data which we actually want. To do that we need to find the boundaries of this slice. We can see in the beginning 500 char, it starts with "context" the starting position would be 2 characters behind the beginning of the word 'context'

In [ ]:
start = script_data.find("context")-2

the ending position would be 12 characters from the end. After that patch this sliced text into the json loads function, to get a python dictionary back

In [ ]:
json_data = json.loads(script_data[start:-12])

In [ ]:
# Using the keys method, it could be found where data for financial statements is loacted in the context, dispacher, stores and QuoteSummaryStore
json_data['context'].keys()

In [ ]:
json_data['context']['dispatcher']['stores']['QuoteSummaryStore'].keys()

# New section